<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-and-Data-Import" data-toc-modified-id="Setup-and-Data-Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup and Data Import</a></span><ul class="toc-item"><li><span><a href="#Exploring" data-toc-modified-id="Exploring-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Exploring</a></span></li><li><span><a href="#Organizing" data-toc-modified-id="Organizing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Organizing</a></span></li></ul></li><li><span><a href="#Cleaning-Missing-Values" data-toc-modified-id="Cleaning-Missing-Values-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cleaning Missing Values</a></span><ul class="toc-item"><li><span><a href="#Imputing-Values-Not-Actually-Missing" data-toc-modified-id="Imputing-Values-Not-Actually-Missing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Imputing Values Not Actually Missing</a></span></li><li><span><a href="#Imputing-Missing-Values" data-toc-modified-id="Imputing-Missing-Values-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Imputing Missing Values</a></span></li><li><span><a href="#Imputing-Lot-Frontage" data-toc-modified-id="Imputing-Lot-Frontage-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Imputing Lot Frontage</a></span></li></ul></li></ul></div>

## Setup and Data Import

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

pd.options.display.max_rows = None

In [2]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

### Exploring

In [3]:
def exploring(df):
    print('Dataframe shape:', df.shape)
    print('\nColumns:\n', df.columns)

In [4]:
exploring(train)

Dataframe shape: (1460, 81)

Columns:
 Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'Garage

In [5]:
exploring(test)

Dataframe shape: (1459, 80)

Columns:
 Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'Garage

### Organizing

In [6]:
# Grouping the features by topic
# addons = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
#                       'PoolQC']
# basement = list(df.columns[df.columns.str.contains('Bsmt')])

# exterior = ['RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
#                     'ExterQual', 'ExterCond', 'Foundation']
# garage = list(df.columns[df.columns.str.contains('Garage')])

# general = ['MSSubClass', 'BldgType', 'HouseStyle', 'YearBuilt', 'YearRemodAdd', 'MiscFeature',
#                    'MiscVal']
# interior = ['1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath',
#                     'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd']
# lot = ['LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig',
#                'LandSlope', 'PavedDrive', 'Fence']
# neighborhood = ['MSZoning', 'Neighborhood', 'Condition1', 'Condition2']

# overall = ['OverallQual', 'OverallCond', 'Functional']

# sale = ['MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice']

# sqft = list(df.columns[df.columns.str.contains('SF')])

# utilities = ['Utilities', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'Fireplaces',
#                      'FireplaceQu']

## Cleaning Missing Values

In [7]:
def missing_val_info(df):
    print('Number of rows with NaN:', len(df[df.isna().any(axis=1)]), '\n')
    cols_na = df.loc[:, df.isna().any()] # df with only columns that have missing values
    print('Columns with NaN:\n', cols_na.isna().sum())

In [8]:
missing_val_info(train)

Number of rows with NaN: 1460 

Columns with NaN:
 LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


In [9]:
missing_val_info(test)

Number of rows with NaN: 1459 

Columns with NaN:
 MSZoning           4
LotFrontage      227
Alley           1352
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinSF1         1
BsmtFinType2      42
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageCars         1
GarageArea         1
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
SaleType           1
dtype: int64


### Imputing Values Not Actually Missing

In [10]:
# Function that can be called with list of column names to fill NaNs with 'N/A' string
def impute(df, cols):
    df[cols] = df[cols].fillna('N/A')

# If observation has TotalBsmtSF > 0, NaN in any Bsmt column != No Basement
def impute_bsmt(df, cols):
    # Store rows where NaN != No Basement
    to_drop = df[cols][(df[cols].isna().any(axis=1)) & (df[cols].TotalBsmtSF > 0)]
    # After dropping rows, fill NaNs
    dframe = df[cols].drop(to_drop.index).fillna('N/A')
    df[cols] = pd.concat([dframe, to_drop])

In [11]:
impute(train, ['Alley', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
               'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
               'Fence', 'MiscFeature'])
impute_bsmt(train, list(train.columns[train.columns.str.contains('Bsmt')]))

In [12]:
impute(test, ['Alley', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
               'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
               'Fence', 'MiscFeature'])
impute_bsmt(test, list(test.columns[test.columns.str.contains('Bsmt')]))

### Imputing Missing Values

In [13]:
def impute_misc_cols(df):
    # MAR? Imputed as None since the same observations are also NaN in MasVnrArea (mostly 0 values)
    df.MasVnrType.fillna('None', inplace=True)
    # MAR? Imputed as 0.0 since the same observations are also NaN in MasVnrType (mostly None values)
    df.MasVnrArea.fillna(0.0, inplace=True)

    # Likely MCAR. Imputed with most common value that was not N/A (which indicates no basement)
    df.BsmtExposure.fillna('No', inplace=True)

    # MCAR? BsmtFinType2 is missing but BsmtFinSF2 has a value, meaning BsmtFinType2 should have a rating.
    df.BsmtFinType2.fillna('Unf', inplace=True)

    # MAR? Some signs this house varies from others in the neighborhood built in the
    # same year, but seems likely it has the same wiring anyway, given that the most
    # similar homes AND the vast majority of homes in the dataset have SBrkr.
    df.Electrical.fillna('SBrkr', inplace=True)

In [14]:
impute_misc_cols(train)
impute_misc_cols(test)

### Imputing Lot Frontage

In [15]:
# No obvious influence, potential relationships with other variables should be explored.
# Also unlikely value should be 0; the lowest value in LotFrontage is 21.
print('LotFrontage missing:', len(train[train.LotFrontage.isna()]))

print('LotFrontage min:', train.LotFrontage.value_counts().index.min())
print('LotFrontage max:', train.LotFrontage.value_counts().index.max())
print('LotFrontage top 5:\n', train.LotFrontage.value_counts().head())

LotFrontage missing: 259
LotFrontage min: 21.0
LotFrontage max: 313.0
LotFrontage top 5:
 60.0    143
70.0     70
80.0     69
50.0     57
75.0     53
Name: LotFrontage, dtype: int64


In [26]:
y_train = train.LotFrontage.dropna()
y_test = test.LotFrontage.dropna()
y_test = y_test.sample(len(y_train), random_state=0)

def dummies_for_X(df):
    lot_config = pd.get_dummies(df.LotConfig
                               ).drop(df.LotConfig.value_counts().index[0], 1)
    zoning = pd.get_dummies(df.MSZoning
                           ).drop(df.MSZoning.value_counts().index[0], 1)
    hood = pd.get_dummies(df.Neighborhood
                         ).drop(df.Neighborhood.value_counts().index[0], 1)
    yr_built = pd.get_dummies(df[df.YearBuilt > 1950
                                ].YearBuilt.value_counts().index[0]
                             ).drop(df.YearBuilt.value_counts().index[0], 1)
    X = pd.concat([lot_config, zoning, hood, yr_built], axis=1)
    X = X.sample(len(y_train), random_state=42)
    return X

X_train = dummies_for_X(train)
X_test = dummies_for_X(test)

In [27]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)


-0.18003791683403825